In [7]:
torch.save(model.state_dict(), "../models/emotion_cnn.pth")

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from models.cnn_model import EmotionCNN
from utils.preprocess import get_dataloaders

# Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader, test_loader, class_names = get_dataloaders()

# Initialize model
model = EmotionCNN(num_classes=7).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {running_loss:.4f} | Accuracy: {acc:.2f}%")


Epoch 1/10 | Loss: 1472.4146 | Accuracy: 35.02%
Epoch 2/10 | Loss: 1306.5891 | Accuracy: 44.01%
Epoch 3/10 | Loss: 1223.8229 | Accuracy: 47.44%
Epoch 4/10 | Loss: 1160.3353 | Accuracy: 50.33%
Epoch 5/10 | Loss: 1107.2230 | Accuracy: 53.24%
Epoch 6/10 | Loss: 1053.4591 | Accuracy: 55.02%
Epoch 7/10 | Loss: 1002.9128 | Accuracy: 57.55%
Epoch 8/10 | Loss: 952.2237 | Accuracy: 60.05%
Epoch 9/10 | Loss: 901.5456 | Accuracy: 62.36%
Epoch 10/10 | Loss: 856.3678 | Accuracy: 64.12%


In [1]:
from utils.preprocess import get_dataloaders

train_loader, test_loader, class_names = get_dataloaders()
print(class_names)  # ['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral']


['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
